# 构建多模态 LLaVA 模型

本教程演示如何基于 LLaVA 架构构建多模态模型，通过结合语言模型和视觉模型以及投影层。LLaVA 模型可以同时处理文本和图像，支持图像描述、视觉问答等任务。

## 什么是 LLaVA？

LLaVA 是一种多模态模型架构，它将视觉编码器与语言模型连接起来，使模型能够同时处理视觉和文本信息。这使得模型能够基于图像输入来理解和生成文本。

## 前提条件

在开始之前，请确保您已安装 ``align-anything`` 包。

```bash
# 克隆仓库
git clone git@github.com:PKU-Alignment/align-anything.git
cd align-anything

# 使用conda创建虚拟环境
conda create -n align-anything python==3.11
conda activate align-anything
```

- **`[Optional]`** We recommend installing [CUDA](https://anaconda.org/nvidia/cuda) in the conda environment and set the environment variable.

```bash
# 我们在 H800 计算集群上测试过，这个版本的 CUDA 效果很好。
# 您可以根据计算集群的实际情况调整此版本。

conda install nvidia/label/cuda-12.2.0::cuda
export CUDA_HOME=$CONDA_PREFIX
```

> 如果您的 CUDA 安装在不同的位置，例如 `/usr/local/cuda/bin/nvcc`，您可以按如下方式设置环境变量：

```bash
export CUDA_HOME="/usr/local/cuda"
```

最后，通过以下命令安装 `align-anything`：

```bash
# 我们为训练和评估准备了快速安装。
# 如果您只需要使用训练或评估模块，
# 您可以安装相应的依赖项。
pip install -e .[train] # 安装训练依赖项
pip install -e .[evaluate] # 安装评估依赖项

# 如果您需要安装所有依赖项，可以使用以下命令：
pip install -e .[all]
```

## 导入所需的库

In [ ]:
import os
import torch
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    CLIPImageProcessor,
    LlavaConfig,
    LlavaForConditionalGeneration,
    LlavaProcessor,
)

## 模型优化的辅助函数

以下函数确保所有模型参数在内存中是连续的，这可以提高性能。

In [2]:
def make_model_contiguous(module):
    """Make all model parameters contiguous in memory for better performance."""
    for child in module.children():
        make_model_contiguous(child)

    for param in module.parameters(recurse=False):
        param.data = param.data.contiguous()

## 设置模型路径

您可以根据自己的偏好自定义这些路径。默认情况下，我们将使用 Meta-Llama-3.1-8B-Instruct 作为语言模型，CLIP ViT-Large 作为视觉模型。

In [3]:
# 定义模型路径
language_model_path = "/PATH/TO/YOUR/Llama-3.1-8B-Instruct"  # 您可以更改此路径以使用任何兼容的语言模型
vision_tower_path = "/PATH/TO/YOUR/clip-vit-large-patch14-336"  # 您可以更改此路径以使用任何兼容的视觉模型
# TODO: 更改为自己的路径
save_path = "/PATH/TO/YOUR/llama_vision"  # 保存组合模型的位置

# 如果目录不存在，则创建目录
os.makedirs(save_path, exist_ok=True)

## 加载基础模型

首先，我们将加载视觉模型、语言模型、分词器和图像处理器。

In [ ]:
# 加载视觉模型
print("Loading vision model...")
vision_model = AutoModel.from_pretrained(vision_tower_path)

# 加载语言模型
print("Loading language model...")
language_model = AutoModelForCausalLM.from_pretrained(language_model_path)

# 加载语言模型的分词器
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(language_model_path)

# 加载视觉模型的图像处理器
print("Loading image processor...")
image_processor = CLIPImageProcessor.from_pretrained(vision_tower_path)

## 准备分词器

我们需要在分词器中添加特殊token以处理图像和填充。

In [ ]:
# 在分词器中添加特殊token
tokenizer.add_special_tokens({'additional_special_tokens': ['<image>', '<unk>', '<pad>']})
tokenizer.pad_token = '<pad>'
tokenizer.unk_token = '<unk>'

print(f"Tokenizer vocabulary size after adding special tokens: {len(tokenizer)}")

## 创建 LLaVA 配置参数

现在我们将创建一个结合视觉和语言模型配置的配置参数。

In [30]:
# 获取视觉模型和语言模型的配置参数
vision_config = vision_model.vision_model.config
language_config = language_model.config

# 创建一个结合视觉和语言模型配置的配置参数
config = LlavaConfig(vision_config, language_config)

# 在配置参数中设置图像token索引
config.image_token_index = tokenizer.convert_tokens_to_ids('<image>')

# 将聊天模板设置为原始 LLaVA 模板
# 从 https://huggingface.co/llava-hf/llava-1.5-7b-hf/blob/main/chat_template.json 复制
llava_template = """"{% for message in messages %}{% if message['role'] != 'system' %}{{ message['role'].upper() + ': '}}{% endif %}{# Render all images first #}{% for content in message['content'] | selectattr('type', 'equalto', 'image') %}{{ '<image>\n' }}{% endfor %}{# Render all text next #}{% if message['role'] != 'assistant' %}{% for content in message['content'] | selectattr('type', 'equalto', 'text') %}{{ content['text'] + ' '}}{% endfor %}{% else %}{% for content in message['content'] | selectattr('type', 'equalto', 'text') %}{% generation %}{{ content['text'] + ' '}}{% endgeneration %}{% endfor %}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ 'ASSISTANT:' }}{% endif %}"""

# 创建一个结合图像处理器和分词器的处理器
processor = LlavaProcessor(
    image_processor=image_processor, 
    tokenizer=tokenizer,
    patch_size=14,
    chat_template=llava_template
)

## 构建 LLaVA 模型

现在我们将通过结合视觉和语言模型来创建 LLaVA 模型。

In [ ]:
# 使用结合的配置参数初始化 LLaVA 模型
print("Building LLaVA model...")
model = LlavaForConditionalGeneration(config)

# 分配预训练的语言模型
model.language_model = language_model

# 分配预训练的视觉模型
model.vision_tower = vision_model

# 调整令牌嵌入以匹配新的分词器大小
model.resize_token_embeddings(len(tokenizer))

# 使模型参数连续以获得更好的性能
make_model_contiguous(model)

print("LLaVA model built successfully!")

## 保存模型

最后，我们将保存组合模型和处理器到磁盘。

在保存模型之前，您可能需要指定 `CUDA_HOME` 环境变量以避免 CUDA 警告。

In [32]:
import os
os.environ["CUDA_HOME"] = "/PATH/TO/YOUR/CUDA" # run `which nvcc` to get the path

In [ ]:
# 保存模型和处理器
print(f"Saving model to {save_path}...")
model.save_pretrained(save_path)
processor.save_pretrained(save_path)
print("Model and processor saved successfully!")

## 测试模型

让我们用一个示例图像测试我们新创建的 LLaVA 模型。

In [34]:
from PIL import Image
import requests

conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "What are these?"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
raw_image = Image.open(requests.get(image_file, stream=True).raw)

你可以先检查图像和提示词。

In [ ]:
prompt

In [ ]:
raw_image

然后，我们加载模型和处理器。

In [ ]:
# 加载保存的模型和处理器
loaded_model = LlavaForConditionalGeneration.from_pretrained(save_path)
loaded_processor = LlavaProcessor.from_pretrained(save_path)

现在我们应该处理图像和提示词到 torch 张量。

In [ ]:
inputs = loaded_processor(images=raw_image, text=prompt, return_tensors='pt')
print('The input has the following keys: ', inputs.keys())
print('Their shapes are: ', {k: v.shape for k, v in inputs.items()})

最后，我们可以生成回答。

In [ ]:
output = loaded_model.generate(**inputs, max_new_tokens=20, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

如你所见，模型的输出非常混乱。这是因为模型没有在图像-文本对上进行训练。根据 LLaVA 论文中的介绍，我们应该在图像-文本对上微调模型以获得更好的性能。一个参考脚本在 `scripts/llava_step1.sh` 文件中提供。

# 模态扩展

训练设置主要基于LLaVA论文。

**注意**：您应该完成以下步骤以确保训练过程顺利进行。

1. 通过以下命令下载`liuhaotian/LLaVA-Pretrain`数据集：

```bash
huggingface-cli download --repo-type dataset --resume-download liuhaotian/LLaVA-Pretrain --local-dir /PATH/TO/YOUR/liuhaotian/LLaVA-Pretrain --local-dir-use-symlinks False
```

2. 解压`images.zip`文件，并将`images`文件夹放在`/PATH/TO/YOUR/images/`目录下。

3. 将环境变量`COCO_DATA_DIR`指定为`images`文件夹的路径。

最终脚本如下：


```bash
export COCO_DATA_DIR="/PATH/TO/YOUR/images/"

# Initialize variables
MODEL_NAME_OR_PATH="/PATH/TO/YOUR/MLLMs"
TRAIN_DATASETS="/PATH/TO/YOUR/liuhaotian/LLaVA-Pretrain"
TRAIN_TEMPLATE="LLaVA_Pretrain" # dataset template
TRAIN_DATA_FILES="blip_laion_cc_sbu_558k.json"

OUTPUT_DIR="../outputs/llava_step1" # output dir

# For wandb online logging
export WANDB_API_KEY=""

# Source the setup script
source ./setup.sh

# Execute deepspeed command
deepspeed \
        --master_port ${MASTER_PORT} \
        --module align_anything.trainers.text_image_to_text.sft \
        --model_name_or_path ${MODEL_NAME_OR_PATH} \
        --train_datasets ${TRAIN_DATASETS} \
        --train_template ${TRAIN_TEMPLATE} \
        --train_split train \
        --train_data_files ${TRAIN_DATA_FILES} \
        --output_dir ${OUTPUT_DIR} \
        --save_total_limit 3 \
        --freeze_vision_tower True \
        --freeze_mm_proj False \
        --freeze_language_model True \
        --epochs 1 \
        --ds_cfgs ds_z2_config.json \
        --learning_rate 1.e-3 \
        --per_device_train_batch_size 16 \
        --gradient_accumulation_steps 32
```





## 结论

恭喜！您已成功通过结合语言模型和视觉模型构建了一个多模态LLaVA模型。该模型现在可以处理文本和图像，从而实现各种多模态应用。

### 致谢

- [Hugging Face Transformers 文档](https://huggingface.co/docs/transformers/index)
- [LLaVA 论文](https://arxiv.org/abs/2304.08485)